Use this utlity to update the returns and std_dev fields within investment-options.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [3]:
# Set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = False

In [5]:
# Read in sp400.csv
sp400 = pd.read_csv('sp400.csv')
sp400.drop(columns=['SEC filings'], inplace=True)
sp400.rename(columns={'Security':'Description',
                      'GICS Sector':'Asset Class',
                      'GICS Sub-Industry': 'GICS Sub Industry'}, inplace=True)
sp400.set_index("Symbol", inplace=True)
sp400

,Description,Asset Class,GICS Sub Industry,Headquarters Location
Symbol,,,,
AA,Alcoa,Materials,Aluminum,"Pittsburgh, Pennsylvania"
AAL,American Airlines Group,Industrials,Passenger Airlines,"Fort Worth, Texas"
AAON,AAON,Industrials,Building Products,"Tulsa, Oklahoma"
ACHC,Acadia Healthcare,Health Care,Health Care Facilities,"Franklin, Tennessee"
ACM,AECOM,Industrials,Construction & Engineering,"Dallas, Texas"
...,...,...,...,...
XPO,"XPO, Inc.",Industrials,Cargo Ground Transportation,"Greenwich, Connecticut"
XRAY,Dentsply Sirona,Health Care,Health Care Supplies,"Charlotte, North Carolina"
YETI,Yeti Holdings,Consumer Discretionary,Leisure Products,"Austin, Texas"


In [6]:
# Read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Energy': 'US Stocks:Energy',
 'Materials': 'US Stocks:Materials',
 'Industrials': 'US Stocks:Industrials',
 'Consumer Discretionary': 'US Stocks:Consumer Discretionary',
 'Consumer Staples': 'US Stocks:Consumer Staples',
 'Health Care': 'US Stocks:Healthcare',
 'Financials': 'US Stocks:Financials',
 'Information Technology': 'US Stocks:Technology',
 'Communication Services': 'US Stocks:Communication Services',
 'Utilities': 'US Stocks:Utilities',
 'Real Estate': 'US Stocks:Real Estate'}

In [7]:
# Map sp400 GICS sectors to brownbear defined asset classes.
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

sp400['Asset Class'] = sp400.apply(_asset_class, axis=1)

# Yahoo finance uses '-' where '.' is used in symbol names.
sp400.index = sp400.index.str.replace('.', '-', regex=False)
sp400

,Description,Asset Class,GICS Sub Industry,Headquarters Location
Symbol,,,,
AA,Alcoa,US Stocks:Materials,Aluminum,"Pittsburgh, Pennsylvania"
AAL,American Airlines Group,US Stocks:Industrials,Passenger Airlines,"Fort Worth, Texas"
AAON,AAON,US Stocks:Industrials,Building Products,"Tulsa, Oklahoma"
ACHC,Acadia Healthcare,US Stocks:Healthcare,Health Care Facilities,"Franklin, Tennessee"
ACM,AECOM,US Stocks:Industrials,Construction & Engineering,"Dallas, Texas"
...,...,...,...,...
XPO,"XPO, Inc.",US Stocks:Industrials,Cargo Ground Transportation,"Greenwich, Connecticut"
XRAY,Dentsply Sirona,US Stocks:Healthcare,Health Care Supplies,"Charlotte, North Carolina"
YETI,Yeti Holdings,US Stocks:Consumer Discretionary,Leisure Products,"Austin, Texas"


In [8]:
# Drop invalid symbols.
sp400.drop(['NYCB', 'PNM'], inplace=True)


In [9]:
# Make symbols list.
symbols = list(sp400.index)
#symbols

In [10]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

AA AAL AAON ACHC ACM ADC AFG AGCO AIT ALE ALGM ALLY ALTM ALTR ALV AM AMED AMG AMH AMKR AN ANF APPF AR ARMK ARW ARWR ASB ASGN ASH ATR AVNT AVT AVTR AXTA AYI AZPN BC BCO BDC BERY BHF BILL BIO BJ BKH BLD BLKB BMRN BRBR BRKR BRX BURL BWXT BYD CACI CADE CAR CASY CBSH CBT CC CCK CDP CELH CFR CG CGNX CHDN CHE CHH CHRD CHWY CHX CIEN CIVI CLF CLH CMC CNH CNM CNO CNX CNXC COHR COKE COLB COLM COTY CPRI CR CRI CROX CRUS CSL CUBE CUZ CVLT CW CXT CYTK DAR DBX DCI DINO DKS DLB DOCS DOCU DT DTM DUOL EEFT EGP EHC ELF ELS EME ENOV ENS ENSG EPR EQH ESAB ESNT EVR EWBC EXE EXEL EXLS EXP EXPO FAF FBIN FCFS FCN FFIN FHI FHN FIVE FIX FLEX FLO FLR FLS FN FNB FND FNF FOUR FR FYBR G GAP GATX GBCI GEF GGG GHC GLPI GME GMED GNTX GPK GT GTLS GXO H HAE HALO HGV HLI HLNE HOG HOMB HQY HR HRB HWC HXL IBKR IBOC IDA ILMN INGR IPGP IRDM IRT ITT JAZZ JEF JHG JLL JWN KBH KBR KD KEX KMPR KNF KNSL KNX KRC KRG LAD LAMR LANC LEA LECO LFUS LITE LIVN LNTH LNW LOPE LPX LSCC LSTR M MAN MANH MASI MAT MEDP MIDD MKSI MLI MMS MORN MSA 

In [11]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df = df[:]
df

,AA,AAL,AAON,ACHC,ACM,ADC,AFG,AGCO,AIT,ALE,...,WTRG,WTS,WU,WWD,X,XPO,XRAY,YETI,ZI,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,36.03,51.08,13.68,59.98,29.49,20.61,30.89,36.31,37.95,38.39,...,21.06,57.25,11.10,45.40,24.51,14.05,46.82,NaN,NaN,22.21
2015-01-05,33.94,51.05,13.22,59.12,28.19,20.85,30.48,34.64,36.99,37.78,...,20.66,55.80,11.00,44.42,23.37,13.62,46.50,NaN,NaN,21.38
2015-01-06,34.19,50.26,13.01,58.19,27.91,20.98,30.31,34.48,36.46,37.75,...,20.68,54.74,10.98,44.10,22.66,13.14,45.92,NaN,NaN,20.56
2015-01-07,35.07,50.23,13.13,60.63,28.48,21.22,30.51,34.45,36.07,38.46,...,20.87,54.33,10.87,44.51,22.71,13.19,47.11,NaN,NaN,20.76
2015-01-08,36.07,50.84,13.45,61.76,29.37,21.30,31.06,35.19,36.27,39.22,...,21.00,54.90,11.09,44.29,23.21,13.26,48.28,NaN,NaN,21.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-26,38.35,17.35,122.68,38.07,108.33,70.55,138.16,93.63,244.44,64.61,...,36.61,206.87,10.69,171.68,31.00,133.17,19.00,39.38,11.06,55.35
2024-12-27,37.68,17.35,121.42,37.96,107.42,70.05,137.00,93.20,241.55,64.43,...,36.46,203.72,10.68,169.91,31.14,132.26,18.94,39.35,10.66,54.68
2024-12-30,37.15,17.62,119.65,38.58,106.79,69.66,136.04,93.10,238.66,64.63,...,36.22,202.90,10.57,168.13,31.03,132.59,18.65,38.64,10.40,54.32


In [12]:
# Sample symbol.
symbol = 'ACHC'

In [13]:
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns[symbol]

np.float64(-47.90442199733956)

In [14]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [15]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola[symbol]

np.float64(0.23264778375209008)

In [16]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola[symbol]

np.float64(0.12109898033433268)

In [23]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('ME').ffill()

In [24]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()
monthly_returns[symbol]

Date
2015-01-31     NaN
2015-02-28    0.12
2015-03-31    0.01
2015-04-30    0.05
2015-05-31    0.02
              ... 
2024-09-30   -0.05
2024-10-31    0.10
2024-11-30    0.16
2024-12-31   -0.10
2025-01-31   -0.00
Freq: ME, Name: ZION, Length: 121, dtype: float64

In [25]:
# Calculate 3 year annualized standard deviation.
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev[symbol]

np.float64(0.45550935192872805)

In [26]:
# Read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
#lines

In [27]:
# For each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = lines.copy()

# This is still slow (2.53 s).
for i, (index, row) in enumerate(sp400.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append(
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
        .format(symbol, description, asset_class,
                ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [28]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')